In [73]:
#| echo: false
import numpy as np
from scipy.interpolate import PchipInterpolator
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Span, Range1d
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
import warnings
import pandas as pd
import altair as alt
warnings.filterwarnings('ignore')
output_notebook()
alt.data_transformers.disable_max_rows()
data = np.loadtxt('data/PT_wafer2_after_d_diodes.dat', delimiter=',')

def plot_format(plot, xlabel, ylabel, location, size, titlesize, labelsize):
    # x axis format
    plot.xaxis.axis_label = xlabel
    plot.xaxis.axis_label_text_font_style = 'bold'
    plot.xaxis.major_label_text_font_style = "bold"
    plot.xaxis.axis_label_text_font_size = size
    plot.xaxis.major_label_text_font_size = size

    # y axis format
    plot.yaxis.axis_label = ylabel
    plot.yaxis.axis_label_text_font_style = 'bold'
    plot.yaxis.major_label_text_font_style = "bold"
    plot.yaxis.axis_label_text_font_size = size
    plot.yaxis.major_label_text_font_size = size

    # Legend format
    plot.legend.location = location
    plot.legend.click_policy = "hide"
    plot.legend.label_text_font_size = labelsize
    plot.legend.label_text_font_style = 'bold'
    plot.legend.border_line_width = 3
    plot.legend.border_line_color = "navy"
    plot.legend.border_line_alpha = 0.0
    plot.legend.background_fill_alpha = 0.0
    plot.legend.label_text_color = "#E3F4FF"


    # Title format
    plot.title.text_font_size = titlesize
    plot.title.text_font_style = "bold"

    # Dark theme
    plot.background_fill_color = "#282B30"
    plot.border_fill_color = "#282B30"
    plot.xgrid.grid_line_color = '#606773'
    plot.ygrid.grid_line_color = '#606773'
    plot.yaxis.major_label_text_color = "#E3F4FF"
    plot.xaxis.major_label_text_color = "#E3F4FF"
    plot.yaxis.axis_label_text_color = "#E3F4FF"
    plot.xaxis.axis_label_text_color = "#E3F4FF"
    plot.title.text_color = "#A6DDFF"
    return plot

Loading BokehJS ...

## Base function

### Different $\mu$ sampling points 

In [74]:
#| echo: false
new_colors = []
for i in range(42):
        new_colors.append('#9D6C97')
        new_colors.append('#9DC3E6')
        new_colors.append('#9DD9C5')

In [4]:
#| column: page

# 1. Read the Excel file into a DataFrame
df = pd.read_excel('data/base_function.xlsx', sheet_name=['base', 'M'])

# 2. Split the DataFrame into two separate DataFrames
base_df = df['base']
M_df = df['M'].sort_values(by='M')
sorted_df = pd.DataFrame(columns=['mu','xaxis', 'yaxis', 'colors'])

# 3. Create x axis
xaxis = np.arange(-15.5, 16.5, 1)
plots = []

# 4. Iterate M dataframe
for i, (index, row) in enumerate(M_df.iterrows()):
    # a. Plot raw sampling data
    p = figure(title=str(f'M: {row.M}'), x_axis_label='sampling point', y_axis_label='intensity', tooltips = [("index", "$index"),("(x,y)", "($x, $y)")],
            width = 250, height = 150)
    new_axis = xaxis - row.M
    p.line(new_axis, base_df[index], line_color='#9DD9C5', line_width=3)
    p.circle(new_axis, base_df[index], size = 4)
    vline = Span(location=0.0, dimension = 'height', line_color='#FEEED9', line_width=1)
    p.add_layout(vline)

    # b. Plot format
    p.x_range = Range1d(-7, 7)
    p.yaxis.ticker.desired_num_ticks = 4
    p = plot_format(p, "Degrees", "Intensity", "bottom_left", "8pt", "8pt", "8pt")
    plots.append(p)

    # c. Create dataframe
    sorted_df = sorted_df.append(pd.DataFrame({'mu':[row.M]*32,'xaxis':new_axis, 'yaxis':base_df[index], 'colors':new_colors[0:32]}), ignore_index=True)
    
    # interleaved_plot.circle(new_axis, base_df[index], color=new_colors[i])

grid_raw = gridplot(children = plots, ncols = 6, merge_tools=False)
show(grid_raw)


### Base function interpolation

In [5]:
#| column: page
# 5. Create interleaved plot
interleaved_plot = figure(title='Interleaved base function', x_axis_label='sampling point', y_axis_label='intensity', tooltips = [("index", "$index"),("(x,y)", "($x, $y)")],
            width = 700, height = 500)
sorted_df_a = sorted_df.sort_values(by='xaxis')

# a. line plot
source = ColumnDataSource(data = sorted_df_a)
interleaved_plot.line(x='xaxis', y='yaxis', source=source, line_width = 6, legend = 'Interleaved curve')

# b. individual points
for (index, row) in sorted_df_a.iterrows():
    interleaved_plot.circle(row.xaxis, row.yaxis, color = row.colors, size = 6)

# c. format
interleaved_plot.xaxis.ticker.desired_num_ticks = 20
interleaved_plot = plot_format(interleaved_plot, "Degrees", "Intensity", "top_left", "10pt", "10pt", "10pt")

# d. Interpolation
interpolated_plot = figure(title='Inteporlated base function points', tooltips = [("index", "$index"),("(x,y)", "($x, $y)")],
            width = 700, height = 500)
 
xinterp = np.arange(-15.5, 15.5001, 0.001)
pchip = PchipInterpolator(sorted_df_a['xaxis'], sorted_df_a['yaxis'])
yinterp = pchip(xinterp)

interpolated_plot.line(x='xaxis', y='yaxis', source=source, line_width = 6, legend = 'Interleaved curve')
interpolated_plot.line(xinterp, yinterp, line_width = 5, color = '#9DD9C5', legend = 'Interpolated base function')
interpolated_plot = plot_format(interpolated_plot, "Degrees", "Intensity", "top_left", "10pt", "10pt", "10pt")
interleaved_plot.xaxis.ticker.desired_num_ticks = 20

# plot
base_function_grid = gridplot(children=[interleaved_plot, interpolated_plot], ncols =2, merge_tools=False, width=600, height = 450)
show(base_function_grid)

## Wafer 2 with rough areas 

### Calcute Aq, M and I values

In [6]:
lendata = data.shape[0] - 1
angles = (np.linspace(0, (lendata - 1), lendata)) * (data[-1, 29] - data[-1, 28]) * (2 * np.pi / lendata / 360) + (data[-1, 28] + 9.5 + 180) * np.pi / 180
radii = (np.linspace(0, (lendata-1), lendata)) * (data[-1,31]-data[-1,30])/lendata + data[-1, 30]

Aq = []
M = []
intens = []
x = np.arange(1, 33)
datamat = data[:lendata,:] - 1

sx = np.arange(1, 32, 0.2)
for indx in range(lendata):
    # a. Do PChip spline  interpolation
    diodes = datamat[indx,:]
    pchip = PchipInterpolator(x, diodes)
    sy = pchip(sx)
    sy = 100 * sy / np.max(sy)
    c = []

    # b. Concatenation/histogram
    for indx2 in range(len(sx)):
        c.extend([indx2+1]*round(sy[indx2]))

    stddev = np.std(c) / 5
    Aq.append(1.02 * np.exp(1.987 * np.log(stddev) + 0.16))
    M.append((np.mean(c) - 1) / 5 - 15.5)
    intens.append(np.sum(sy))

### Non interpolated Plot

In [7]:
# 1. Get x and y coordinates
xvals = radii * np.cos(angles) * 1e-3
yvals = radii * np.sin(angles) * 1e-3
zvals = Aq

# 2. Clamp values to max of 3
zvalscut = 3
zvalsnew = np.array(zvals)
zvalsnew[zvalsnew > zvalscut] = zvalscut

# 3. Make a df
index = np.arange(len(xvals))
df = pd.DataFrame({
    'index':index,
    'x': xvals*1000,
    'y': yvals*1000,
    'z': zvalsnew,   
})

#| column: screen-inset-right
# 4. Rank_text
interval = alt.selection_interval()
rank_text = alt.Chart(df).mark_text(dx=20, dy=-5, align='left').encode(
    x=alt.value(10),  # x position of the text
    y=alt.value(10),  # y position of the text
    text=alt.condition(
        interval,
        alt.Text('index:Q', format='.2f'),  # display 'x' column for selected points
        alt.value('')  # display empty string for unselected points
    ),
    color=alt.value('black'), # color of the text
    fontSize = 5
)

# 5. Wafer plot
wafer_plot = alt.Chart(df).mark_circle().encode(
    x='x:Q', y='y:Q',
    color=alt.condition(
        interval,
        alt.Color('z:Q', scale=alt.Scale(scheme='turbo')),
        alt.value('lightgray')
    ),
    tooltip=['x', 'y', 'z', 'index']
).properties(height=400, width=400).add_selection(interval)

# create a table with the selected points
selected_points_table = alt.Chart(df).transform_filter(
    interval
).mark_text().encode(
    x=alt.value(0),
    y=alt.Y('row_number:O', axis=None),
    text='index:Q',
).transform_window(
    row_number='row_number()'
).properties(
    height=400,
    width=50
)

# vertically concatenate the two charts
alt.hconcat(wafer_plot, selected_points_table)



alt.HConcatChart(...)

## Rougher data

In [72]:
from bokeh.palettes import Set3
# 1. Import data
rough_df = pd.read_excel('data/rough_samples.xlsx')
source_rough = ColumnDataSource(rough_df)

# 2. Create plot
rough_plot = figure(x_axis_label='xaxis', y_axis_label='yaxis', width = 650, height = 400, tooltips = [("index", "$index"),("(x,y)", "($x, $y)")])
color_palette = Set3[len(rough_df.columns[1:])+1]
rough_plot.line(xinterp, yinterp, line_width=4, color = color_palette[i+1], legend = 'Interpolated base function')

# a. iterate over the columns and add a line for each one
for i, col in enumerate(rough_df.columns[1:]):
    rough_plot.line('xaxis', col, source=source_rough, color = color_palette[i], legend_label = str(col), line_width=4)
    rough_plot.circle('xaxis', col, source=source_rough, color = color_palette[i], size=6)

rough_plot = plot_format(rough_plot, "Degrees", "Intensity", "top_left", "10pt", "10pt", "10pt")

show(rough_plot)

In [ ]:
#| echo: false
# # Read the Excel file into a DataFrame
# df = pd.read_excel('data/base_function.xlsx', sheet_name=['base', 'M'])

# # Split the DataFrame into two separate DataFrames
# base_df = df['base']
# M_df = df['M'].sort_values(by='M')

# # Create x axis
# xaxis = np.arange(-15.5, 16.5, 1)

# # Iterate M dataframe
# fig, axs = plt.subplots(nrows=4, ncols=6, figsize=(12,8))
# axs = axs.flatten()

# for i, (index, row) in enumerate(M_df.iterrows()):
#     # Calculate shifted axis
#     new_axis = xaxis - row.M
#     # Plot base function
#     axs[i].plot(new_axis, base_df[index], color='#9DD9C5')
#     axs[i].scatter(new_axis, base_df[index], s=15, color='#9DD9C5')
#     # Add vertical line
#     axs[i].axvline(x=0, color='#FEEED9', linewidth=1)
#     # Set x and y axis labels
#     axs[i].set_xlabel('sampling point')
#     axs[i].set_ylabel('intensity')
#     # Set title and tooltip
#     axs[i].set_title(f'M: {row.M}')
#     axs[i].set_xlim(-16, 16)
#     axs[i].set_ylim(0, 40000)
#     # Add grid
#     axs[i].grid(True, linestyle='--', alpha=0.6)
    
# # Adjust layout
# plt.tight_layout()
# plt.show()

In [ ]:
#| echo: false
# zvalscut_hi = 2.04
# zvalscut_low = 1.96
# zvals = Aq
# zvalsnew = np.copy(zvals)
# zvalsnew[zvalsnew > zvalscut_hi] = zvalscut_hi
# zvalsnew[zvalsnew < zvalscut_low] = zvalscut_low

# matindx = []
# xaxis = np.arange(-55, 55.5, 0.5) * 1e-3
# yaxis = np.arange(-55, 55.5, 0.5) * 1e-3
# regmatAq = np.zeros((len(xaxis), len(yaxis)))
# nummat = np.zeros((len(xaxis), len(yaxis)))

# for xindx in range(len(xaxis)):
#     for yindx in range(len(yaxis)):
#         d = np.column_stack((xaxis[xindx] - xvals, yaxis[yindx] - yvals))
#         e = np.sqrt(d[:, 0]**2 + d[:, 1]**2)
#         dindx = np.where(e < 0.0008)[0]
#         if len(dindx)>0:
#             regmatAq[xindx, yindx] = np.mean(zvalsnew[dindx])
#         else:
#             regmatAq[xindx, yindx] = np.nan

# regmatAq[0:2, 0] = [zvalscut_hi, zvalscut_low]

In [ ]:
#| echo: false
# # Create x and y coordinate arrays
# x, y = np.meshgrid(xaxis * 1e-3, yaxis * 1e-3)
# z = regmatAq
# source = pd.DataFrame({'x': x.ravel(),
#                      'y': y.ravel(),
#                      'z': z.ravel()})

# wafer_plot_interpolated = alt.Chart(source).mark_rect().encode(
#     x=alt.X('x:O', axis=None),
#     y=alt.Y('y:O', axis=None),
#     color=alt.Color('z:Q', scale=alt.Scale(scheme='turbo'))
# ).properties(width=400, height=400).interactive()

# wafer_plot_interpolated

In [ ]:
#| echo: false
# zvals = M
# df = pd.DataFrame({
#     'index':index,
#     'x': xvals*1000,
#     'y': yvals*1000,
#     'z': zvals,   
# })

# alt.Chart(df).mark_circle().encode(
#     x='x:Q', y='y:Q',
#     color= alt.Color('z:Q', scale=alt.Scale(scheme='Viridis'),
    
#     ),
#     tooltip=['x', 'y', 'z', 'index']
# ).properties(height=400, width=400).interactive()


In [ ]:
#| echo: false
# zvals = M
# zvalscut=0.2
# zvalsnew= np.array(zvals)
# zvalsnew[np.abs(zvalsnew) > zvalscut] = zvalscut

# df = pd.DataFrame({
#     'index':index,
#     'x': xvals*1000,
#     'y': yvals*1000,
#     'z': zvalsnew,   
# })

# alt.Chart(df).mark_circle().encode(
#     x='x:Q', y='y:Q',
#     color= alt.Color('z:Q', scale=alt.Scale(scheme='Turbo'),
    
#     ),
#     tooltip=['x', 'y', 'z', 'index']
# ).properties(height=400, width=400).interactive()


In [ ]:
sorted_df